<a href="https://colab.research.google.com/github/shobhit8797/Hollywood_Actors_Network_Visualizer/blob/main/DataPrepForKumu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install scikit-network==0.24.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.2 MB 15.6 MB/s 


In [3]:
!rm *.tsv.gz
!wget -q https://datasets.imdbws.com/name.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.principals.tsv.gz
!wget -q https://datasets.imdbws.com/title.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.akas.tsv.gz
!ls -la

rm: cannot remove '*.tsv.gz': No such file or directory
total 1079508
drwxr-xr-x 1 root root      4096 Nov 17 17:51 .
drwxr-xr-x 1 root root      4096 Nov 17 17:51 ..
drwxr-xr-x 4 root root      4096 Nov 15 14:30 .config
-rw-r--r-- 1 root root 236472510 Nov 16 13:19 name.basics.tsv.gz
drwxr-xr-x 1 root root      4096 Nov 15 14:31 sample_data
-rw-r--r-- 1 root root 287598573 Nov 16 13:19 title.akas.tsv.gz
-rw-r--r-- 1 root root 163905072 Nov 16 13:19 title.basics.tsv.gz
-rw-r--r-- 1 root root 417407164 Nov 16 13:19 title.principals.tsv.gz


In [4]:
# load the titles
title = pd.read_csv('title.basics.tsv.gz', sep='\t', low_memory=True).set_index('tconst')[['titleType','primaryTitle','startYear']]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
title.head()

,titleType,primaryTitle,startYear
tconst,,,
tt0000001,short,Carmencita,1894
tt0000002,short,Le clown et ses chiens,1892
tt0000003,short,Pauvre Pierrot,1892
tt0000004,short,Un bon bock,1892
tt0000005,short,Blacksmith Scene,1893


In [6]:
title.dtypes

titleType       object
primaryTitle    object
startYear       object
dtype: object

In [7]:
title = title[title['startYear'] >= '2000'] # filter by year

In [8]:
title.head()

,titleType,primaryTitle,startYear
tconst,,,
tt0011801,movie,Tötet nicht mehr,2019
tt0013274,movie,Istoriya grazhdanskoy voyny,2021
tt0015414,movie,La tierra de los toros,2000
tt0021123,movie,The March of Time,\N
tt0034413,short,Youth Gets a Break,2001


In [9]:
# load the cast of each film
cast  = pd.read_csv('title.principals.tsv.gz', sep = '\t')[['tconst', 'nconst', 'category']]
# only consider actors, not directors, composors etc.
cast = cast[cast.category.isin({'actor','actress'})]
cast.head()

,tconst,nconst,category
11,tt0000005,nm0443482,actor
12,tt0000005,nm0653042,actor
16,tt0000007,nm0179163,actor
17,tt0000007,nm0183947,actor
21,tt0000008,nm0653028,actor


In [10]:
# only consider movies, not series, etc
movies = title[title['titleType'] == 'movie']
cast = cast[cast['tconst'].isin(movies.index)]
cast.head()

,tconst,nconst,category
80694,tt0011801,nm0459029,actor
80695,tt0011801,nm0681726,actor
80696,tt0011801,nm0692612,actress
80697,tt0011801,nm0726256,actor
80698,tt0011801,nm0776458,actor


In [11]:
region = pd.read_csv('title.akas.tsv.gz',sep='\t',usecols=['titleId','region'],low_memory = True,dtype={'region':'str','titleId':'str'}).set_index('titleId')['region']
region.value_counts().head(10)

JP    4041630
FR    4040455
DE    4037623
IN    3973695
ES    3962847
IT    3943446
PT    3878470
\N    1869323
US    1368456
GB     424571
Name: region, dtype: int64

In [12]:
region.head()

titleId
tt0000001    UA
tt0000001    DE
tt0000001    HU
tt0000001    GR
tt0000001    RU
Name: region, dtype: object

In [13]:
name = pd.read_csv('name.basics.tsv.gz', sep = '\t', low_memory=True, na_values = '\\N', dtype={'birthYear': float}).set_index('nconst')[['primaryName', 'birthYear']]
name.head()

,primaryName,birthYear
nconst,,
nm0000001,Fred Astaire,1899.0
nm0000002,Lauren Bacall,1924.0
nm0000003,Brigitte Bardot,1934.0
nm0000004,John Belushi,1949.0
nm0000005,Ingmar Bergman,1918.0


In [14]:
from scipy.sparse import csr_matrix

In [15]:
def get_pairs(lang=None, min_acted=25, min_pairings=1):
    '''
    Returns an adjacency matrix and actor mapping of actor pairs where:
    - Each actor has acted in at least min_acted films
    - The two actors have acted together in at least min_pairings films
    - And (optionally), belong to a region `lang` (IN, UN, etc)
    '''
    graph = cast
    if lang is not None:
        graph = graph[graph['tconst'].isin(region[region == lang].index)]
        # graph = graph[graph['tconst'].isin(region[region == lang].index)]
    name_freq = graph['nconst'].value_counts()
    top_names = name_freq[name_freq >= min_acted]
    top_actors = graph[graph['nconst'].isin(top_names.index)]

    p = top_actors.copy()
    p['title'] = p['tconst'].astype('category')
    p['name'] = p['nconst'].astype('category')

    row = p['title'].cat.codes.values
    col = p['name'].cat.codes.values
    data = np.ones(len(p), dtype='int')

    matrix = csr_matrix((data, (row, col)))
    square = matrix.T * matrix
    square.setdiag(0)
    square = square.tocoo()

    pairs = pd.DataFrame({
        'row': square.row,
        'col': square.col,
        'n': square.data
    })
    pairs = pairs[pairs.n >= min_pairings].reset_index(drop=True)
    return pairs, name.reindex(p['name'].cat.categories)

def lookup(pairs, cat):
    pairs = pd.concat([
        pairs,
        cat.iloc[pairs.row].reset_index(drop=True),
        cat.iloc[pairs.col].reset_index(drop=True),
    ], axis=1)
    pairs = pairs.drop(columns=['row', 'col'])
    pairs.columns = ['count', 'name1', 'year1', 'name2', 'year2']
    return pairs.sort_values('count', ascending=False)

In [16]:
pairs, cat = get_pairs(lang='US', min_acted=1, min_pairings=1) # setting lang = 'IN' will apply a filter allowing only INdian movies

In [17]:
ForKumu = lookup(pairs, cat)

In [18]:
ForKumu = ForKumu[['name1', 'name2', 'count']]
ForKumu = ForKumu.rename(columns={'name1':'From',
                                  'name2':'To',
                                  'count':'Strength'})


In [19]:
ForKumu.head()

,From,To,Strength
141404,Kevin Murphy,Bill Corbett,51
292454,Michael J. Nelson,Kevin Murphy,51
141403,Michael J. Nelson,Bill Corbett,51
296433,Bill Corbett,Michael J. Nelson,51
296432,Kevin Murphy,Michael J. Nelson,51


In [20]:
ForKumu.shape

(1790892, 3)

In [21]:
ForKumu.head(10000).to_excel("Hollywood_2000_excel.xlsx", index = False)